In [0]:
# https://medium.com/@move37timm/using-kaggle-api-for-google-colaboratory-d18645f93648
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth, files

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
import tensorflow as tf
#from tensorflow import keras
import keras
import numpy as np
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

# Test if we have a GPU
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Using TensorFlow backend.


Found GPU at: /device:GPU:0


## Download data from Kaggle.

In [0]:
# Get data from kaggle
# https://medium.com/@move37timm/using-kaggle-api-for-google-colaboratory-d18645f93648
!pip install kaggle

# Copy + pasted from medium article:

auth.authenticate_user()

drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])

# Had to change this from /content to /root.
filename = "/root/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)

request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

# API call from Kaggle
!kaggle competitions download -c mnist-digit-classification-2


    100% |████████████████████████████████| 61kB 1.2MB/s 
    100% |████████████████████████████████| 235kB 3.5MB/s 
  Running setup.py bdist_wheel for kaggle ... - \ done
  Stored in directory: /root/.cache/pip/wheels/8b/21/3b/a0076243c6ae12a6215b2da515fe06b539aee7217b406e510e
  Running setup.py bdist_wheel for python-slugify ... - done
  Stored in directory: /root/.cache/pip/wheels/e3/65/da/2045deea3098ed7471eca0e2460cfbd3fdfe8c1d6fa6fcac92
Successfully built kaggle python-slugify
Download 100%.
  0% 0.00/77.1k [00:00<?, ?B/s]
100% 77.1k/77.1k [00:00<00:00, 34.4MB/s]
  0% 0.00/2.12M [00:00<?, ?B/s]
100% 2.12M/2.12M [00:00<00:00, 51.6MB/s]
 78% 10.0M/12.8M [00:00<00:00, 12.3MB/s]
100% 12.8M/12.8M [00:00<00:00, 17.9MB/s]


## Read MNIST training data, split for training/testing

In [0]:
SEED=5387
# Grab data.
df_train = pd.read_csv('/content/mnist_train.csv.zip', compression='zip',
                      index_col=0)
df_test = pd.read_csv('/content/mnist_test.csv.zip', compression='zip',
                      index_col=0)
df_sample = pd.read_csv('/content/mnist_sample.csv', index_col=0)
#print('Training data:')
#print(df_train.head(10))
#print(df_train.describe())
#print('Testing data:')
#print(df_test.head(3))
x = df_train.drop(['Category'], axis=1).values

# Normalize x.
x = x / 255.0
df_test = df_test/255.0

# Reshape for convolutional network. Convolutional network stuff from:
# https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
img_rows, img_cols = 28, 28

#np.reshape(x, (x.shape[0], 28, 28))

if K.image_data_format() == 'channels_first':
    x_conv = np.reshape(x, (x.shape[0], 1, img_rows, img_cols))
    x_final = np.reshape(df_test.values, (df_test.shape[0], 1, img_rows, img_cols))
    input_shape = (1, img_rows, img_cols)
else:
    x_conv = np.reshape(x, (x.shape[0], img_rows, img_cols, 1))
    input_shape = (img_rows, img_cols, 1)
    x_final = np.reshape(df_test.values, (df_test.shape[0], img_rows, img_cols, 1))
y = df_train['Category'].values
# print(x_train.describe())

# Split data.
# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25,
#                                                     random_state=SEED)
x_train, x_test, y_train, y_test = train_test_split(x_conv, y, test_size=0.25,
                                                    random_state=SEED)

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)
y = keras.utils.to_categorical(y, 10)

## PCA

In [0]:
# PCA_DIM = 150
# pca = PCA(n_components=PCA_DIM)
# # Do PCA with all the data
# pca.fit(x)
# x_pca = pca.transform(x)
# x_pca_train = pca.transform(x_train)
# x_pca_test = pca.transform(x_test)
# print(x_pca_train.shape)

## Train Neural Network

In [0]:
# Build neural network model. For now, 1 hidden layer.
# Input has a neuron for each PCA dimension, hidden layer
# has half the neurons of the input, and the output is 
# our predictor.

early_stop = keras.callbacks.EarlyStopping(monitor='acc', patience=3)

# model = keras.Sequential([
#    keras.layers.Dense(PCA_DIM, input_shape=(PCA_DIM,), activation=tf.nn.relu),
#    keras.layers.Dense(PCA_DIM, activation=tf.nn.relu),
#    keras.layers.Dense(PCA_DIM*2/3, activation=tf.nn.relu),
#    keras.layers.Dense(PCA_DIM/3, activation=tf.nn.relu),
#    keras.layers.Dense(10, activation=tf.nn.softmax)
# ])

# model.compile(optimizer=tf.train.AdamOptimizer(),
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
# model.fit(x_pca_train, y_train, epochs=100, callbacks=[early_stop])

model = Sequential()
model.add(Conv2D(64, kernel_size=(4, 4),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(128, (4, 4), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              #optimizer=tf.train.AdadeltaOptimizer(),
              metrics=['accuracy'])

model.fit(x_conv, y, epochs=100, batch_size=1000)

Epoch 1/100
60000/60000 [==============================] - 13s 224us/step - loss: 0.7102 - acc: 0.7846
Epoch 2/100
60000/60000 [==============================] - 12s 197us/step - loss: 0.1626 - acc: 0.9520
Epoch 3/100
60000/60000 [==============================] - 12s 198us/step - loss: 0.1076 - acc: 0.9675
Epoch 4/100
60000/60000 [==============================] - 12s 196us/step - loss: 0.0821 - acc: 0.9759
Epoch 5/100
60000/60000 [==============================] - 12s 196us/step - loss: 0.0644 - acc: 0.9805
Epoch 6/100
60000/60000 [==============================] - 12s 197us/step - loss: 0.0586 - acc: 0.9828
Epoch 7/100
60000/60000 [==============================] - 12s 196us/step - loss: 0.0493 - acc: 0.9854
Epoch 8/100
60000/60000 [==============================] - 12s 196us/step - loss: 0.0429 - acc: 0.9862
Epoch 9/100
60000/60000 [==============================] - 12s 196us/step - loss: 0.0400 - acc: 0.9877
Epoch 10/100
60000/60000 [==============================] - 12s 197us/ste

## Test Neural Network, Retrain with All Data

In [0]:
# test_loss, test_acc = model.evaluate(x_pca_test, y_test)
# print('Test accuracy:', test_acc)
# 97.3 w/ PCA @ 100 features, 10 epochs
# 97.6 w/ PCA @ 150 features, 10 epochs
# 97.4 w/ PCA @ 150 features, 10 epochs, simple network w/ one hidden layer (mean of input/output)
# 97.4 w/ PCA @ 150 features, 10 epochs, two hidden layers (mean, then "4-mean")
# 97.4 w/ PCA @ 150 features, 10 epochs, two hidden layers, both size (PCA + 10)/2
# 97.3 w/ PCA @ 150 features, 10 epochs, three hidden layers, all size (PCA + 10)/2
# 97.1 w/ PCA @ 200 features, 10 epochs, two hidden layers, both size (PCA + 10)/2 
# 97.3 w/ PCA @ 200 features, 10 epochs, three hidden layers: 0.75, 0.5, 0.25 * PCA
# 97.2 w/ PCA @ 125 features, 14 epochs, three hidden layers: 0.75, 0.5, 0.25 * PCA
# 97.5 w/ PCA @ 150 features, 20 epochs, three hidden layers: 0.75, 0.25, 0.25 * PCA
# 97.4 w/ PCA @ 150 features, 21 epochs, two hidden layers: 2/3, 1/3 PCA
# 97.2 w/ PCA @ 150 features, 16 epochs, two hidden layers @ PCA
# 97.5 w/ PCA @ 150 features, 22 epochs, three hidden layers: PCA, 2/3, 1/3
# 97.67 w/ PCA @ 150 features, 21 epochs, four hidden layers: PCA (specified 
#       input dimension), PCA, 2/3, 1/3
# model.fit(x_pca, y, epochs=100, callbacks=[early_stop])

## Predict, Write to File

In [0]:
#y_pred = np.argmax(model.predict(pca.transform(df_test.values)), axis=1)
y_pred = np.argmax(model.predict(x_final), axis=1)
df_sample['Category'] = y_pred
df_sample.to_csv('mnist.csv')
files.download('mnist.csv')